## Notebook to draw links between quotebank and our movie dataset

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [9]:
year = "2017"
#movie crew
df1 = pd.read_csv("movie_data_2015_2020.csv")
df1.head()
df1 = df1[df1["category"]!="archive_footage"]
df1 = df1[df1["characters"]!="[\"Self\"]"]

In [10]:
def process_chunk(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    print(chunk.columns)
    #here you can filter etc
 
# read df chunk by chunk, long! 
with pd.read_json(f'../../data/quotes-{year}.json.bz2', lines=True, compression='bz2', chunksize=1000000) as df_reader:
    pbar = tqdm(enumerate(df_reader))
    for i,chunk in pbar:
        if i == 0:
            quotes = chunk[chunk["speaker"].isin(df1["primaryName"])]
        else:
            quotes = pd.concat((quotes,chunk[chunk["speaker"].isin(df1["primaryName"])]))
        pbar.set_description(f"Number of quotes {quotes.shape[0]}")

Number of quotes 651241: : 27it [26:14, 58.31s/it]


In [11]:
print(f"**{quotes.shape[0]} citations founds :)**")

**651241 citations founds :)**


In [12]:
quotes.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
23,2017-11-02-082415,"I've just started a production company, so I'm...",Maisie Williams,[Q234363],2017-11-02 08:26:49,4,"[[Maisie Williams, 0.8412], [None, 0.1408], [R...",[http://www.nme.com/news/tv/maisie-williams-ga...,E
34,2017-11-24-000160,"[ In ] Dia de los Muertos, the whole point is ...",Lee Unkrich,[Q380920],2017-11-24 09:00:14,1,"[[Lee Unkrich, 0.9533], [None, 0.0467]]",[http://www.slashfilm.com/cocos-family/],E
58,2017-07-19-000396,[ Producers ] said Saru is the Spock of the se...,Doug Jones,"[Q1251779, Q16142774, Q30122825, Q461309, Q530...",2017-07-19 17:07:28,2,"[[Doug Jones, 0.8193], [None, 0.1807]]",[http://comicbook.com/2017/07/19/star-trek-dis...,E
93,2017-06-26-058719,"Jackie keeps Quan well-grounded, Pierce keeps ...",Martin Campbell,"[Q1445998, Q282033]",2017-06-26 11:10:18,1,"[[Martin Campbell, 0.8054], [None, 0.0986], [J...",[http://rssfeeds.usatoday.com/~/376286728/0/us...,E
137,2017-06-20-067359,Joe's folks are afraid Taylor is going to swal...,Joe Alwyn,[Q27440303],2017-06-20 00:27:00,1,"[[Joe Alwyn, 0.5531], [None, 0.3135], [Cedric ...",[http://jezebel.com/kim-kardashians-lessons-le...,E


In [13]:
quotes.to_csv(f"movie_{year}_crew_quotes.csv.gz",compression='gzip')

In [14]:
quotes.groupby("speaker").agg(count=("speaker","count")).sort_values(["count"],ascending=False)

,count
speaker,
Jimmy Kimmel,8421
Roy Moore,5747
John Fox,4282
Selena Gomez,3926
Kyrie Irving,3810
...,...
Ally Ioannides,1
Andrew Hollander,1
Gonzalo Maza,1


In [8]:
df1.shape

(9999, 39)

In [5]:
df1[df1["primaryName"]=="Kobe Bryant"]

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
9261,9261,tt7935892,movie,The Oscar Nominated Short Films 2018: Live Action,The Oscar Nominated Short Films 2018: Live Action,0.0,2018.0,NaN,96.0,"Action,Drama",...,1.0,nm1101483,actor,NaN,NaN,Kobe Bryant,1978.0,2020.0,"producer,writer,actor","tt7935892,tt1442437,tt1528854,tt6794476"


In [17]:
df1[df1["category"]=="archive_footage"]

,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,...,ordering_y,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
1300,1300,tt2870648,movie,Amy,Amy,0.0,2015.0,NaN,128.0,"Biography,Documentary,Music",...,1.0,nm1561881,archive_footage,NaN,"[""Self""]",Amy Winehouse,1983.0,2011.0,"soundtrack,composer,music_department","tt5164214,tt2194499,tt1343092,tt2870648"
1303,1303,tt2870648,movie,Amy,Amy,0.0,2015.0,NaN,128.0,"Biography,Documentary,Music",...,4.0,nm1258970,archive_footage,NaN,"[""Self""]",Russell Brand,1975.0,NaN,"actor,producer,writer","tt1336608,tt1334512,tt1226229,tt0800039"
1312,1312,tt6322922,movie,An Inconvenient Sequel: Truth to Power,An Inconvenient Sequel: Truth to Power,0.0,2017.0,NaN,98.0,Documentary,...,4.0,nm0001051,archive_footage,NaN,"[""Self""]",Bill Clinton,1946.0,NaN,"producer,actor,writer","tt12660626,tt7689964,tt0108515,tt7466880"
1528,1528,tt8760684,movie,Apollo 11,Apollo 11,0.0,2019.0,NaN,93.0,"Documentary,History",...,1.0,nm0035842,archive_footage,NaN,"[""Self - Mission Commander""]",Neil Armstrong,1930.0,2012.0,"actor,camera_department","tt0101886,tt0418795,tt0312305,tt14747578"
1529,1529,tt8760684,movie,Apollo 11,Apollo 11,0.0,2019.0,NaN,93.0,"Documentary,History",...,2.0,nm0172531,archive_footage,NaN,"[""Self - Command Module Pilot""]",Michael Collins,1930.0,2021.0,"actor,cinematographer,camera_department","tt0925248,tt9395794,tt0403702,tt8760684"
1530,1530,tt8760684,movie,Apollo 11,Apollo 11,0.0,2019.0,NaN,93.0,"Documentary,History",...,3.0,nm0004696,archive_footage,NaN,"[""Self - Lunar Module Pilot""]",Buzz Aldrin,1930.0,NaN,"actor,miscellaneous,writer","tt1399103,tt1872220,tt0898266,tt8760684"
1531,1531,tt8760684,movie,Apollo 11,Apollo 11,0.0,2019.0,NaN,93.0,"Documentary,History",...,4.0,nm0805654,archive_footage,NaN,"[""Self - Director of Flight Crew Operations""]",Deke Slayton,1924.0,1993.0,writer,"tt0270656,tt0170980,tt0347717,tt0124010"
3339,3339,tt10256238,movie,Created Equal: Clarence Thomas in His Own Words,Created Equal: Clarence Thomas in His Own Words,0.0,2020.0,NaN,116.0,"Biography,Documentary",...,2.0,nm0081182,archive_footage,NaN,"[""Self""]",Joe Biden,1942.0,NaN,actor,"tt0106172,tt9894116,tt1266020,tt8289196"
3340,3340,tt10256238,movie,Created Equal: Clarence Thomas in His Own Words,Created Equal: Clarence Thomas in His Own Words,0.0,2020.0,NaN,116.0,"Biography,Documentary",...,3.0,nm0384043,archive_footage,NaN,"[""Self""]",Anita Hill,1956.0,NaN,NaN,"tt14573272,tt7152912,tt5795282,tt9134666"
3406,3406,tt8574836,movie,Cunningham,Cunningham,0.0,2019.0,NaN,93.0,"Biography,Documentary,Music",...,2.0,nm0128509,archive_footage,NaN,"[""Self""]",John Cage,1912.0,1992.0,"composer,music_department,writer","tt1130884,tt0039340,tt0166191,tt6106076"
